In [2]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [3]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [4]:
#  baseurl = 'https://vast-depths-89393.herokuapp.com/'
#baseurl = 'https://glacial-hollows-90395.herokuapp.com/'
baseurl = 'http://127.0.0.1:5000/'

In [77]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [78]:
response.json()

{'signingKey_hex': '577d933cccf943d098df73f8f15b3f8ac0c1fa718e5b546793547b100ed06dec',
 'traderId': '31',
 'verifyKey_hex': 'c27f7f9b8a2eafc724a33974292d25227eebd15c3144758f3091a2af563739b9'}

In [80]:
url = baseurl + 'createMarket'

mkId = 6
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [81]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [83]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,False,1,1,0,6,31


In [97]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk,
                     "traderId": tId,
                     "verifyKey_hex": vk,
                     "marketId": mkId,
                     "price": 0.55,
                     "quantity":1}

In [100]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [101]:
response.json()

{'checks': 'True',
 'marketId': 6,
 'price': 0.55,
 'quantity': 1,
 'traderId': '31'}

In [103]:
# View json response
pd.DataFrame(response.json(), index=[0])


,checks,marketId,price,quantity,traderId
0,True,6,0.55,1,31


In [104]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [105]:
response.json()

{'checks': 'True',
 'marketId': 6,
 'price': 0.55,
 'quantity': -1,
 'traderId': '31'}

In [106]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 6.082891941070557 seconds.
matched trade: 1 in 6.049079895019531 seconds.
matched trade: 2 in 5.986768007278442 seconds.
matched trade: 3 in 5.998902797698975 seconds.
matched trade: 4 in 5.992459058761597 seconds.
matched trade: 5 in 6.03984522819519 seconds.
matched trade: 6 in 6.253859996795654 seconds.
matched trade: 7 in 6.1270856857299805 seconds.
matched trade: 8 in 6.069535970687866 seconds.
matched trade: 9 in 6.07305383682251 seconds.


In [107]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [108]:
response.json()

{'checks': 'False',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 31}

In [109]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [110]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,0,0
1,6,1,0,1
2,1,2,0,0
3,2,1,0,1
4,3,1,0,1
5,4,1,0,1
6,5,1,0,1


In [111]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [112]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.5000,1,1
1,1,1,0.5000,1,1
10,1,1,0.5000,1,1
11,1,1,0.5000,1,1
12,1,1,0.5000,1,1
13,1,1,0.5000,1,1
14,1,1,0.5000,1,1
15,6,6,0.5653,1,24
16,1,1,0.5000,1,1
17,1,1,0.5000,1,1


In [113]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 3.046639919281006 seconds.
unmatched trade: 1 in 3.0318541526794434 seconds.
unmatched trade: 2 in 3.018264055252075 seconds.
unmatched trade: 3 in 3.0063316822052 seconds.
unmatched trade: 4 in 2.9699618816375732 seconds.
unmatched trade: 5 in 3.033247947692871 seconds.
unmatched trade: 6 in 2.975416898727417 seconds.
unmatched trade: 7 in 3.276021718978882 seconds.
unmatched trade: 8 in 3.0946340560913086 seconds.
unmatched trade: 9 in 3.1344592571258545 seconds.


In [114]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [115]:
pd.read_json(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [116]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [117]:
pd.read_json(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [124]:
tId

'6'

In [2]:
a = [1, 2, 3]
a

[1, 2, 3]

In [4]:
1+2

3